In [1]:
from test_denoiser import test_acc, show_random_batch, evaluate_metrics
from custom_dataset import get_dataloader
from models.denoisers import DnCNN, REDNet20
from utils import create_resnet
import torch
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#mnist
#dncnn
mnist_denoiser1 = DnCNN(in_channels=3, out_channels=3, depth=7, hidden_channels=64, use_bias=False).to(device)
mnist_denoiser1.load_state_dict(torch.load('./trained_denoisers/dncnn_mnist_mixed+gaussian.pth', map_location=device))
#dae denoiser
mnist_denoiser2 = REDNet20(in_channels=3, out_channels=3, use_bias=False).to(device)
mnist_denoiser2.load_state_dict(torch.load('./trained_denoisers/dae_mnist_mixed+gaussian.pth', map_location=device))

#cifar10
#dncnn
cifar10_denoiser1 = DnCNN(in_channels=3, out_channels=3, depth=7, hidden_channels=64, use_bias=False).to(device)
cifar10_denoiser1.load_state_dict(torch.load('./trained_denoisers/dncnn_cifar10_mixed+gaussian.pth', map_location=device))
cifar10_denoiser2 = DnCNN(in_channels=3, out_channels=3, depth=7, hidden_channels=64, use_bias=False).to(device)
cifar10_denoiser2.load_state_dict(torch.load('./trained_denoisers/dncnn_cifar10_mixed.pth', map_location=device))
cifar10_denoiser3 = DnCNN(in_channels=3, out_channels=3, depth=7, hidden_channels=64, use_bias=False).to(device)
cifar10_denoiser3.load_state_dict(torch.load('./trained_denoisers/dncnn_cifar10_gaussian.pth', map_location=device))
#dae
cifar10_denoiser4 = REDNet20(in_channels=3, out_channels=3, use_bias=False).to(device)
cifar10_denoiser4.load_state_dict(torch.load('./trained_denoisers/dae_cifar10_mixed+gaussian.pth', map_location=device))

<All keys matched successfully>

In [2]:
import pickle
import os
import seaborn as sns


def run_attack(test_loader, net, denoisers, attack_mode, eps_range, norm):
    #initialize result
    result = {}
    result['adv'] = []
    for i in range(len(denoisers)):
        result[f'dn{i+1}'] = []
        
    for eps in eps_range:
        print("eps = ", eps)
        accuracies = test_acc(net, denoisers, test_loader, attack_mode, norm, eps)
        result['adv'].append(accuracies[1])
        for i in range(len(denoisers)):
            result[f'dn{i+1}'].append(accuracies[i+2])

    return result

def plot_mnist_result(result, attack, norm, eps_range, savefig=True, denoised2=True):

    acc_list = unpack_dict(result)
    sns.set_theme(style="darkgrid")
    plt.figure(figsize=(8,6))
    plt.plot(eps_range, acc_list[0], marker='o', linewidth=1.5, markersize=3, label="No Denoiser")
    plt.plot(eps_range, acc_list[1], marker='o', linewidth=1.5, markersize=3, label="DnCNN - mixed")

    # if denoised2 and self_compare:
    #     plt.plot(eps_range, acc_list[2], marker='o', linewidth=1.5, markersize=3, label="DnCNN - gaussian")
    #     plt.plot(eps_range, acc_list[2], marker='o', linewidth=1.5, markersize=3, label="DnCNN - fgsm + pgd")
        
    if denoised2:
        plt.plot(eps_range, acc_list[2], marker='o', linewidth=1.5, markersize=3, label="AutoEncoder - mixed")

    plt.legend(loc="upper right")
    plt.xlabel('epsilon')
    plt.ylabel('model accuracy')
    plt.title(f"l-{norm} {attack} attack on MNIST")
    if savefig:
        if not os.path.exists('denosier_attack_figures'): os.mkdir('denosier_attack_figures')
        fig_name = f"mnist_{attack}_l{norm}.png"
        plt.savefig(os.path.join('denosier_attack_figures', fig_name), dpi=500)

    plt.show()

def plot_cifar10_result(result, attack, norm, eps_range, savefig=True):

    acc_list = unpack_dict(result)
    sns.set_theme(style="darkgrid")
    plt.figure(figsize=(8,6))
    plt.plot(eps_range, acc_list[0], marker='o', linewidth=1.5, markersize=3, label="No Denoiser")
    plt.plot(eps_range, acc_list[1], marker='o', linewidth=1.5, markersize=3, label="DnCNN - fgsm+pgd+gaussian")
    plt.plot(eps_range, acc_list[2], marker='o', linewidth=1.5, markersize=3, label="DnCNN - fgsm+pgd")
    plt.plot(eps_range, acc_list[3], marker='o', linewidth=1.5, markersize=3, label="DnCNN - gaussian")
    plt.plot(eps_range, acc_list[4], marker='o', linewidth=1.5, markersize=3, label="AutoEncoder - fgsm+pgd+gaussian")

    plt.legend(loc="upper right")
    plt.xlabel('epsilon')
    plt.ylabel('model accuracy')
    plt.title(f"l-{norm} {attack} attack on CIFAR10")
    if savefig:
        if not os.path.exists('denosier_attack_figures'): os.mkdir('denosier_attack_figures')
        fig_name = f"cifar10_{attack}_l{norm}.png"
        plt.savefig(os.path.join('denosier_attack_figures', fig_name), dpi=500)

    plt.show()
    

def save_result(filename, result, root='denoiser_attack_results', overwrite=False):
    if not os.path.exists(root): os.mkdir(root)

    file_path = os.path.join(root, filename)
    if os.path.exists(file_path) and (not overwrite):
        print("Updating pickle file")
        prev_result = load_result(file_path)
        prev_result.update(result)
        result = prev_result
        
    with open(file_path, 'wb') as f:
        pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)


def load_result(file_path):
    if not os.path.exists(file_path):
        raise ValueError("Results file not found.")
    
    with open(file_path, "rb") as f:
        result = pickle.load(f)
    
    return result


def unpack_dict(result_dict):
    acc_list = []
    for key in result_dict:
        acc = result_dict[key]
        acc_list.append(acc)
    return acc_list


def load_model_data(dataset):
    #vanilla resnet model
    if dataset == 'mnist':
        resnet = create_resnet(device=device, grayscale=True)
        resnet.load_state_dict(torch.load('./trained_models/mnist/resnet18_2.0+0_BL.pth', map_location=device))
        #dataset loader
        train_loader, test_loader = get_dataloader(dataset='mnist', batch_size=64, sample_test=True)
    elif dataset == 'cifar10':
        resnet = create_resnet(device=device, grayscale=False)
        resnet.load_state_dict(torch.load('./trained_models/cifar10/resnet18_2.0+0_BL.pth', map_location=device))

        train_loader, test_loader = get_dataloader(dataset='cifar10', batch_size=64, sample_test=True)

    print(len(train_loader.dataset))
    print(len(test_loader.dataset))

    return resnet, train_loader, test_loader

    
def run_denoiser_exp(dataset, denoisers, attacks=['fgsm', 'pgd', 'cw']):

    print(f"Running experiment on {dataset} ...")
    # load pretrained resnet and data
    net, _, test_loader = load_model_data(dataset)
    #l2 attack eps
    l2_eps = [0.5, 1.0, 2.0, 3.5]
    #linf attack eps
    linf_eps = [x/256 for x in [2, 5, 10, 16]]

    # run attack
    for attack in attacks:
        if attack == 'cw':
            print(f"\nEvaluating {attack} attack ...")
            file_name = f"{dataset}_{attack}.pkl"
            # l2 norm cw attack
            result = run_attack(test_loader, net, denoisers, attack, l2_eps, norm=2)
            save_result(file_name, result, overwrite=True)
        else:
            # l2 and linf norm for fgsm and pgd
            for norm in [2, np.inf]:
                print(f"\nEvaluating {attack} attack on l{norm} norm ...")
                file_name = f"{dataset}_{attack}_l{str(norm)}.pkl"
                # get eps range
                eps_range = l2_eps if norm == 2 else linf_eps
                
                result = run_attack(test_loader, net, denoisers, attack, eps_range, norm=norm)
                save_result(file_name, result, overwrite=True)


In [ ]:
# run mnist
mnist_denoisers = [mnist_denoiser1, mnist_denoiser2]
run_denoiser_exp(dataset='mnist', denoisers=mnist_denoisers)

In [3]:
# run cifar10
cifar10_denoisers = [cifar10_denoiser1, cifar10_denoiser2, cifar10_denoiser3, cifar10_denoiser4]
run_denoiser_exp(dataset='cifar10', denoisers=cifar10_denoisers, attacks=['cw'])

Running experiment on cifar10 ...
Files already downloaded and verified
Files already downloaded and verified
50000
1024

Evaluating cw attack ...
eps =  0.5


100%|██████████| 16/16 [04:24<00:00, 16.54s/it]


Test Accuracy no attack: 0.7060546875
Test Accuracy with cw attack: 0.197265625
Test Accuracy with cw attack + denoiser1: 0.669921875
Test Accuracy with cw attack + denoiser2: 0.6611328125
Test Accuracy with cw attack + denoiser3: 0.6474609375
Test Accuracy with cw attack + denoiser4: 0.6494140625
eps =  1.0


100%|██████████| 16/16 [04:22<00:00, 16.40s/it]


Test Accuracy no attack: 0.7060546875
Test Accuracy with cw attack: 0.14453125
Test Accuracy with cw attack + denoiser1: 0.669921875
Test Accuracy with cw attack + denoiser2: 0.6611328125
Test Accuracy with cw attack + denoiser3: 0.6474609375
Test Accuracy with cw attack + denoiser4: 0.6494140625
eps =  2.0


100%|██████████| 16/16 [04:22<00:00, 16.40s/it]


Test Accuracy no attack: 0.7060546875
Test Accuracy with cw attack: 0.14453125
Test Accuracy with cw attack + denoiser1: 0.669921875
Test Accuracy with cw attack + denoiser2: 0.6611328125
Test Accuracy with cw attack + denoiser3: 0.6474609375
Test Accuracy with cw attack + denoiser4: 0.6494140625
eps =  3.5


100%|██████████| 16/16 [04:22<00:00, 16.39s/it]

Test Accuracy no attack: 0.7060546875
Test Accuracy with cw attack: 0.14453125
Test Accuracy with cw attack + denoiser1: 0.669921875
Test Accuracy with cw attack + denoiser2: 0.6611328125
Test Accuracy with cw attack + denoiser3: 0.6474609375
Test Accuracy with cw attack + denoiser4: 0.6494140625


In [ ]:
#plot mnist
for attack in ['fgsm', 'pgd', 'cw']:
    if attack == 'cw':
        file_path = os.path.join('denoiser_attack_results', f"mnist_{attack}.pkl")
        result = load_result(file_path)
        plot_mnist_result(result, attack, '2', eps_range=[0.5, 1.0, 2.0, 3.5], savefig=True)
    else:
        file_path = os.path.join('denoiser_attack_results', f"mnist_{attack}_linf.pkl")
        result = load_result(file_path)
        plot_mnist_result(result, attack, 'inf', eps_range=[x/256 for x in [2, 5, 10, 16]], savefig=True)

In [ ]:
#plot cifar10
for attack in ['fgsm', 'pgd', 'cw']:
    if attack == 'cw':
        file_path = os.path.join('denoiser_attack_results', f"cifar10_{attack}.pkl")
        result = load_result(file_path)
        plot_cifar10_result(result, attack, '2', eps_range=[0.5, 1.0, 2.0, 3.5], savefig=True)
    else:
        #print(f"\nEvaluating {attack} attack on l{norm} norm ...")
        file_path = os.path.join('denoiser_attack_results', f"cifar10_{attack}_linf.pkl")
        result = load_result(file_path)
        plot_cifar10_result(result, attack, 'inf', eps_range=[x/256 for x in [2, 5, 10, 16]], savefig=True)

In [ ]:
net, _, test_loader = load_model_data('cifar10')
evaluate_metrics(net, cifar10_denoiser1, test_loader, attack='fgsm', norm=np.inf, eps=0.0625)
evaluate_metrics(net, cifar10_denoiser1, test_loader, attack='pgd', norm=np.inf, eps=0.0625)

In [ ]:
#show_random_batch(mnist_resnet, mnist_denoiser1, mnist_test_loader, attack='fgsm', norm=np.inf, eps=0.0625)
net, _, test_loader = load_model_data('cifar10')
show_random_batch(net, cifar10_denoiser1, test_loader, attack='fgsm', norm=np.inf, eps=0.0625, eval=True)
show_random_batch(net, cifar10_denoiser1, test_loader, attack='pgd', norm=np.inf, eps=0.0625, eval=True)
# show_random_batch(net, cifar10_denoiser1, test_loader, attack='cw', norm=2, eps=3.5, eval=True)

In [ ]:
file_path = os.path.join('denoiser_attack_results', f"cifar10_fgsm_linf.pkl")
result1 = load_result(file_path)
file_path = os.path.join('denoiser_attack_results', f"cifar10_pgd_linf.pkl")
result2 = load_result(file_path)
list1 = unpack_dict(result1)
list2 = unpack_dict(result2)
l11 = np.array(list1[0])
l12 = np.array(list1[1])
l21 = np.array(list2[0])
l22 = np.array(list2[1])
print(np.mean(l12 - l11))
print(np.mean(l22 - l21))

In [ ]:
import torch
x = torch.ones((4, 4)) / 2
y = torch.ones((2, 3, 4, 4))
yy = torch.randn_like(y[:, 0, :, :].unsqueeze(1))

#print(x)
z = x + y
print(z.size())
#print(z)
print(y.shape[-1])
print(y.shape[-2])
print(yy.shape)

In [ ]:
import torch
x = torch.tensor([-0.2, 0.3, 1.5])
print(torch.clamp(x, 0, 1))

In [ ]:
import numpy as np

a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
print(a-b)
print(np.sum(a-b))

In [ ]:
x = {'a': [1,2], 'b': 2}
y = {'a': [3,4], 'b': 4}
print(x)
x['a'] = y['a']
print(x)

In [ ]:
indices = torch.randperm(10000)[:1000]
print(indices.size())

In [ ]:
acc_list = np.zeros((3))
print(acc_list)
acc_list[0] += 1
print(acc_list)

In [ ]:
for norm in [2, np.inf]: print(str(norm))

In [ ]:
a = torch.tensor([True, False, True])
b = torch.tensor([1, 1, 1])
c = torch.tensor([3, 3, 3])
b[a] = c[a]
print(b)